# Get ASOH Data for All CAMP Cells
We'll use the CAMP data for now

In [5]:
from moirae.interface.hdf5 import HDF5Writer
from moirae.interface import run_online_estimate
from moirae.extractors.ecm import MaxCapacityExtractor, OCVExtractor
from battdat.data import CellDataset
from roviweb.online import load_estimator
from pathlib import Path
from tqdm import tqdm
import numpy as np
import warnings

In [2]:
camp_dir = Path('../../publishing-camp-data/data/hdf5/refined')
estimator_path = Path('../tests/files/example-estimator.py')
write_dir = Path('asoh-estimates')

In [3]:
write_dir.mkdir(exist_ok=True, parents=True)

# Run State Estimation on All
Load a fresh estimator, run estimation storing every cycle, continue

In [4]:
temp_path = Path('temp.asoh.h5')
files = list(camp_dir.rglob('*h5'))
for file in tqdm(files):
    # Skip if already done
    out_path = write_dir / file.with_suffix('.asoh.h5').name
    if out_path.exists():
        continue

    # Estimate the capacity and OCV based on the first few cycles
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        data = CellDataset.from_hdf(file)
    data.tables['raw_data'] = data.raw_data.query('cycle_number < 10')
    q_t = MaxCapacityExtractor().extract(data)

    # Load the estimator and update the initial q_t
    estimator = load_estimator(estimator_path.read_text(), working_dir=estimator_path.parent)
    qt_ind = estimator.state_names.index('q_t.base_values')
    estimator.filter.hidden.mean[qt_ind] = q_t.base_values.item()
     
    # Run and write estimates to temp file
    temp_path.unlink(missing_ok=True)
    writer = HDF5Writer(hdf5_output=temp_path, 
                        per_timestep='mean_var',
                        per_cycle='full',
                        table_options={'complib': 'zlib', 'complevel': 9})
    try:
        run_online_estimate(file, estimator, output_states=False, hdf5_output=writer)
    except np.linalg.LinAlgError:
        continue

    # Copy temp to true
    temp_path.rename(out_path)

/home/lward/miniconda3/envs/roviweb/lib/python3.11/site-packages/battdat/data.py:57: UserWarning: Metadata was created in a different version of battdat. supplied=0.4.0, current=0.4.1.                                              | 0/300 [00:00<?, ?it/s]
  warnings.warn(f'Metadata was created in a different version of battdat. supplied={supplied_version}, current={__version__}.')
/home/lward/miniconda3/envs/roviweb/lib/python3.11/site-packages/battdat/data.py:57: UserWarning: Metadata was created in a different version of battdat. supplied=0.4.0, current=0.4.1.                                  | 1/300 [05:52<29:14:30, 352.08s/it]
  warnings.warn(f'Metadata was created in a different version of battdat. supplied={supplied_version}, current={__version__}.')
/home/lward/miniconda3/envs/roviweb/lib/python3.11/site-packages/battdat/data.py:57: UserWarning: Metadata was created in a different version of battdat. supplied=0.4.0, current=0.4.1.                                  | 2/300 [07:08<1

KeyboardInterrupt: 